In [1]:
import pandas as pd
import json
import sqlite3
from quizzes import process_quiz

db_file = 'progsnap2_21_consenting.db'
NUM_QUIZ_INCLUDE = 10

# Subject table
df_LinkSubject = pd.read_sql(f"""
SELECT * FROM LinkSubject
""", sqlite3.connect(db_file))

# Assignment table
df_LinkAssignment = pd.read_sql(f"""
SELECT * FROM LinkAssignment
""", sqlite3.connect(db_file))

In [2]:
# Get subject ID, 295
list_subjectID = df_LinkSubject.loc[:,'SubjectID']

# Get quiz ID, 384 388 434
list_quizID = df_LinkAssignment.loc[df_LinkAssignment['X-Name'].str.contains('Quiz'), 'AssignmentId']
list_quizID = list_quizID[:NUM_QUIZ_INCLUDE]

In [3]:
# Split students' final grade into high and low based on the median
df_main = df_LinkSubject[['SubjectID', 'X-FinalTotalScore']]
df_main['HighLow'] = (df_main['X-FinalTotalScore'] > df_main['X-FinalTotalScore'].median()).astype(int)
df_main

C:\Users\mianh\AppData\Local\Temp/ipykernel_520/807054103.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_main['HighLow'] = (df_main['X-FinalTotalScore'] > df_main['X-FinalTotalScore'].median()).astype(int)


,SubjectID,X-FinalTotalScore,HighLow
0,630,92.14,1
1,638,97.62,1
2,571,75.32,0
3,575,91.97,1
4,617,65.50,0
...,...,...,...
99,637,97.17,1
100,633,74.80,0
101,815,37.80,0
102,737,24.89,0


In [4]:
# Filter first 10 quizzes, with all subject ID
subject_query = ("'" + list_subjectID + "'").to_string(index=False).replace('\n',',')
quiz_query = ("'" + list_quizID + "'").to_string(index=False).replace('\n',',')
df_quiz_questions = pd.read_sql(f"""
    SELECT SubjectID, AssignmentID, EventType
    FROM MainTable
    WHERE MainTable.CodeStateID != '0'
    AND MainTable.AssignmentID in ({quiz_query})
    AND MainTable.SubjectID in ({subject_query})
    """, sqlite3.connect(db_file))
df_quiz_questions

,SubjectID,AssignmentID,EventType
0,577,384,File.Edit
1,577,384,File.Edit
2,577,384,File.Edit
3,577,384,File.Edit
4,577,384,File.Edit
...,...,...,...
32816,808,345,X-Submission.LMS
32817,808,345,File.Edit
32818,808,345,File.Edit
32819,808,345,File.Edit


In [5]:
# Possible features:
# Number of events for quiz
# Number of submission for quiz
for index in range(NUM_QUIZ_INCLUDE):
    sub_df_quiz_questions = df_quiz_questions.loc[df_quiz_questions['AssignmentID'].str.contains(list_quizID.iloc[index]), ['SubjectID']]
    num_event = sub_df_quiz_questions.value_counts().to_frame()
    num_event.columns = ['Quiz_num_event_'+str(index+1)]
    df_main = pd.merge(df_main, num_event, how='left', on='SubjectID')
    sub_df_quiz_questions = df_quiz_questions.loc[df_quiz_questions['AssignmentID'].str.contains(list_quizID.iloc[index]), ['SubjectID', 'EventType']]
    sub_df_quiz_questions = sub_df_quiz_questions.loc[sub_df_quiz_questions['EventType'].str.contains('X-Submission.LMS'), ['SubjectID']]
    num_submission = sub_df_quiz_questions.value_counts().to_frame()
    num_submission.columns = ['Quiz_num_submission_'+str(index+1)]
    df_main = pd.merge(df_main, num_submission, how='left', on='SubjectID')
df_main = df_main.fillna(0)
df_main

,SubjectID,X-FinalTotalScore,HighLow,Quiz_num_event_1,Quiz_num_submission_1,Quiz_num_event_2,Quiz_num_submission_2,Quiz_num_event_3,Quiz_num_submission_3,Quiz_num_event_4,...,Quiz_num_event_6,Quiz_num_submission_6,Quiz_num_event_7,Quiz_num_submission_7,Quiz_num_event_8,Quiz_num_submission_8,Quiz_num_event_9,Quiz_num_submission_9,Quiz_num_event_10,Quiz_num_submission_10
0,630,92.14,1,31.0,3.0,56.0,11.0,8.0,1.0,11.0,...,41.0,10.0,14.0,2.0,46.0,6.0,18.0,3.0,47.0,8.0
1,638,97.62,1,16.0,2.0,27.0,3.0,10.0,2.0,9.0,...,25.0,2.0,27.0,6.0,29.0,1.0,28.0,5.0,22.0,2.0
2,571,75.32,0,19.0,2.0,24.0,3.0,10.0,2.0,9.0,...,25.0,5.0,33.0,5.0,41.0,4.0,18.0,3.0,23.0,2.0
3,575,91.97,1,13.0,1.0,120.0,21.0,16.0,3.0,9.0,...,12.0,1.0,62.0,12.0,40.0,3.0,17.0,3.0,65.0,13.0
4,617,65.50,0,12.0,1.0,32.0,4.0,10.0,2.0,11.0,...,30.0,7.0,15.0,2.0,32.0,1.0,17.0,2.0,27.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,637,97.17,1,18.0,1.0,30.0,4.0,10.0,2.0,10.0,...,21.0,3.0,79.0,14.0,34.0,2.0,24.0,5.0,58.0,11.0
100,633,74.80,0,16.0,2.0,18.0,2.0,29.0,5.0,9.0,...,15.0,2.0,15.0,3.0,42.0,4.0,62.0,19.0,35.0,5.0
101,815,37.80,0,25.0,3.0,34.0,6.0,17.0,3.0,12.0,...,21.0,2.0,30.0,5.0,38.0,2.0,22.0,2.0,49.0,8.0
102,737,24.89,0,26.0,2.0,60.0,8.0,32.0,5.0,15.0,...,43.0,11.0,39.0,7.0,62.0,6.0,63.0,16.0,104.0,24.0


In [6]:
X = df_main.loc[:, 'Quiz_num_event_1':'Quiz_num_submission_10'].to_numpy()
y = df_main['HighLow'].to_numpy()

In [7]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.gaussian_process.kernels import RBF

In [8]:
names = [
    "Nearest Neighbors",
    "Linear SVM",
    "RBF SVM",
    "Gaussian Process",
    "Decision Tree",
    "Random Forest",
    "Neural Net",
    "AdaBoost",
    "Naive Bayes",
    "QDA",
]

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis(),
]

In [9]:
for name, clf in zip(names, classifiers):
    clf.fit(X, y)
    scores = cross_val_score(clf, X, y, cv=5)
    print(name, scores.mean())

Nearest Neighbors 0.5766666666666665
Linear SVM 0.5576190476190476
RBF SVM 0.4809523809523809
Gaussian Process 0.5671428571428571
Decision Tree 0.5380952380952381
Random Forest 0.5014285714285714


C:\Coding\Anaconda3\envs\cisc320\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


Neural Net 0.5271428571428571
AdaBoost 0.5680952380952381
Naive Bayes 0.529047619047619
QDA 0.5566666666666666
